In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation, metrics
import scipy.stats as ss

/home/yury/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
train = pd.read_csv("orange_small_churn_train_data.csv", index_col='ID')
categors = train.ix[:, 190:230]
categors.head()

,Var191,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
ID,,,,,,,,,,,,,,,,,,,,,
0,NaN,NESt0G8EIb,AERks4l,NaN,taul,1K8T,0LaQ,UaKK0yW,I1sFbv_0IT,NaN,...,Al6ZaUT,vr93T2a,LM8l689qOp,NaN,NaN,fKCe,02N6s8f,xwM2aC7IdeMC0,NaN,NaN
1,NaN,P1WvyxLp3Z,2Knk1KF,NaN,taul,1K8T,YFAj,Bnunsla,o64y9zI,DlISMzi,...,oslk,6hQ9lNX,LM8l689qOp,NaN,ELof,xb3V,RAYp,55YFVY9,mj86,NaN
2,NaN,FoxgUHSK8h,LrdZy8QqgUfkVShG,NaN,taul,1K8T,TyGl,fhk21Ss,nQUveAzAF7,NaN,...,zCkv,catzS2D,LM8l689qOp,NaN,NaN,FSa2,ZI9m,ib5G6X1eUxUn6,mj86,NaN
3,NaN,vNEvyxLp3Z,RO12,NaN,taul,1K8T,0Xwj,uoZk2Zj,LWyxgtXeJL,NaN,...,oslk,e4lqvY0,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN
4,NaN,4e7gUH7IEC,RO12,NaN,taul,1K8T,vSNn,kugYdIL,ZIXKpoNpqq,NaN,...,oslk,MAz3HNj,LM8l689qOp,NaN,NaN,WqMG,RAYp,F2FyR07IdsN7I,NaN,NaN


In [4]:
names_to_drop = []
for name in categors.columns:
    col = categors[name].dropna()
    if len(col.unique()) < 2:
        names_to_drop.append(name)

categors.drop(names_to_drop, axis=1, inplace=True)

categors.fillna(method='bfill', inplace=True)
categors.head()

,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,Var201,...,Var219,Var220,Var221,Var222,Var223,Var225,Var226,Var227,Var228,Var229
ID,,,,,,,,,,,,,,,,,,,,,
0,NESt0G8EIb,AERks4l,SEuy,taul,1K8T,0LaQ,UaKK0yW,I1sFbv_0IT,DlISMzi,smXZ,...,FzaX,hLKtJ9p,Al6ZaUT,vr93T2a,LM8l689qOp,ELof,fKCe,02N6s8f,xwM2aC7IdeMC0,mj86
1,P1WvyxLp3Z,2Knk1KF,SEuy,taul,1K8T,YFAj,Bnunsla,o64y9zI,DlISMzi,smXZ,...,FzaX,hN8KpA1,oslk,6hQ9lNX,LM8l689qOp,ELof,xb3V,RAYp,55YFVY9,mj86
2,FoxgUHSK8h,LrdZy8QqgUfkVShG,SEuy,taul,1K8T,TyGl,fhk21Ss,nQUveAzAF7,feR8rQZ,smXZ,...,FzaX,4UxGlow,zCkv,catzS2D,LM8l689qOp,ELof,FSa2,ZI9m,ib5G6X1eUxUn6,mj86
3,vNEvyxLp3Z,RO12,SEuy,taul,1K8T,0Xwj,uoZk2Zj,LWyxgtXeJL,feR8rQZ,smXZ,...,FzaX,ylCK5YS,oslk,e4lqvY0,LM8l689qOp,ELof,xb3V,RAYp,F2FyR07IdsN7I,am7c
4,4e7gUH7IEC,RO12,SEuy,taul,1K8T,vSNn,kugYdIL,ZIXKpoNpqq,feR8rQZ,smXZ,...,FzaX,PYpzAu9,oslk,MAz3HNj,LM8l689qOp,ELof,WqMG,RAYp,F2FyR07IdsN7I,am7c


In [5]:
y = train['labels']

In [6]:
#преобразуем все признаки в числовые
cat_copy = categors.copy()
for name in cat_copy.columns:
    counts = {}
    pairs = zip(cat_copy[name].values, y)
    for x in pairs:
        if counts.get(x[0]) == None:
                counts[x[0]] = 0
        if x[1] == 1:
            counts[x[0]] += 1
    cat_copy[name] = cat_copy[name].apply(lambda x: counts[x])
    print cat_copy[name][:10]

ID
0    16
1    18
2    29
3     9
4    22
5    20
6    23
7    22
8     8
9    23
Name: Var192, dtype: int64
ID
0     102
1     268
2      10
3    2433
4    2433
5    2433
6    2433
7    2433
8    2433
9    2433
Name: Var193, dtype: int64
ID
0    2935
1    2935
2    2935
3    2935
4    2935
5    2935
6    2935
7    2935
8    2935
9    2935
Name: Var194, dtype: int64
ID
0    2896
1    2896
2    2896
3    2896
4    2896
5    2896
6    2896
7    2896
8    2896
9    2896
Name: Var195, dtype: int64
ID
0    2937
1    2937
2    2937
3    2937
4    2937
5    2937
6    2937
7    2937
8    2937
9    2937
Name: Var196, dtype: int64
ID
0      7
1      9
2    274
3    311
4     15
5      8
6    311
7     17
8    274
9     21
Name: Var197, dtype: int64
ID
0      0
1      0
2    138
3      4
4      0
5      0
6      4
7      4
8      8
9      1
Name: Var198, dtype: int64
ID
0     2
1    10
2     5
3     8
4     2
5     3
6     0
7     4
8    24
9    14
Name: Var199, dtype: int64
ID
0    0
1    0
2  

In [7]:
cat_copy.to_csv("categories_only_as_numbers.csv")

In [8]:
#теперь преобразуем только часть в числовые, а остальные бинаризуем
cat_dums = pd.get_dummies(categors[[x for x in categors.columns if len(categors[x].unique()) < 10]])
cat_dums.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40000 entries, 0 to 39999
Data columns (total 54 columns):
Var194_CTUH          40000 non-null float64
Var194_SEuy          40000 non-null float64
Var194_lvza          40000 non-null float64
Var196_1K8T          40000 non-null float64
Var196_JA1C          40000 non-null float64
Var196_mKeq          40000 non-null float64
Var196_z3mO          40000 non-null float64
Var201_6dX3          40000 non-null float64
Var201_smXZ          40000 non-null float64
Var203_9_Y1          40000 non-null float64
Var203_F3hy          40000 non-null float64
Var203_HLqf          40000 non-null float64
Var203_dgxZ          40000 non-null float64
Var203_pybr          40000 non-null float64
Var205_09_Q          40000 non-null float64
Var205_VpdQ          40000 non-null float64
Var205_sJzTlal       40000 non-null float64
Var208_kIsH          40000 non-null float64
Var208_sBgB          40000 non-null float64
Var210_3av_          40000 non-null float64
Var210_7A3j

In [9]:
cat_copy = categors.copy()
for name in cat_copy.columns:
    if len(cat_copy[name].unique()) >= 10:
        counts = {}
        pairs = zip(cat_copy[name].values, y)
        for x in pairs:
            if counts.get(x[0]) == None:
                    counts[x[0]] = 0
            if x[1] == 1:
                counts[x[0]] += 1
        cat_copy[name] = cat_copy[name].apply(lambda x: counts[x])

cat_copy = cat_copy.drop([x for x in categors.columns if len(categors[x].unique()) < 10], axis=1)
cat_copy.head()

,Var192,Var193,Var195,Var197,Var198,Var199,Var200,Var202,Var204,Var206,Var207,Var212,Var214,Var216,Var217,Var219,Var220,Var222,Var226,Var228
ID,,,,,,,,,,,,,,,,,,,,
0,16,102,2896,7,0,2,0,0,106,1362,65,38,0,11,0,2742,0,0,154,88
1,18,268,2896,9,0,10,0,0,39,110,2334,251,0,4,0,2742,0,0,132,171
2,29,10,2896,274,138,5,1,3,63,118,245,27,1,0,2,2742,138,138,607,85
3,9,2433,2896,311,4,8,1,1,65,1362,2334,2127,1,243,1,2742,4,4,132,2270
4,22,2433,2896,15,0,2,1,0,81,541,2334,2127,1,39,0,2742,0,0,248,2270


In [10]:
all_cats = pd.concat([cat_dums, cat_copy], axis=1)
all_cats.head()

,Var194_CTUH,Var194_SEuy,Var194_lvza,Var196_1K8T,Var196_JA1C,Var196_mKeq,Var196_z3mO,Var201_6dX3,Var201_smXZ,Var203_9_Y1,...,Var207,Var212,Var214,Var216,Var217,Var219,Var220,Var222,Var226,Var228
ID,,,,,,,,,,,,,,,,,,,,,
0,0,1,0,1,0,0,0,0,1,1,...,65,38,0,11,0,2742,0,0,154,88
1,0,1,0,1,0,0,0,0,1,1,...,2334,251,0,4,0,2742,0,0,132,171
2,0,1,0,1,0,0,0,0,1,1,...,245,27,1,0,2,2742,138,138,607,85
3,0,1,0,1,0,0,0,0,1,0,...,2334,2127,1,243,1,2742,4,4,132,2270
4,0,1,0,1,0,0,0,0,1,1,...,2334,2127,1,39,0,2742,0,0,248,2270


In [11]:
all_cats.to_csv("categories_binarized_and_as_numbers.csv")